# Web Scraping of Falcon 9 and Falcon Heavy Launches Records from Wikipedia

We will now web scrape Falcon 9 historical launch records from the Wikipedia page titled `List of Falcon 9 and Falcon Heavy Launches`.

The objective is to use `BeautifulSoup`:
- To extract Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas dataframe

In [1]:
!pip3 install beautifulsoup4

In [2]:
import sys

from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import requests
import numpy

Some helper functions to process web scraped HTML table

In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML table cell
    Input: the element of a table data cell extract extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)[0:2]]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML table cell.
    Input:the element of a table data cell extracts extra row
    """
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell.
    Input: the element of a table data cell extracts extra row'
    """
    out = [i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg")+2]
    else:
        new_mass = 0
    return new_mass

def extract_column_from_header(row):
    """
    This function processes column name from the header row by stripping """
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    column_name = ' '.join(row.contents)
    
    #Filter the digit and empty names
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name

To keep the lab tasks consistent, the data will be scraped from a snapshot of the `List of Falcon 9 and Falcon Heavy Launches` Wikipedia page updated on `9th June 2021`.

In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

## Request the Falcon 9 launch Wiki page from url

In [5]:
response = requests.get(static_url)

In [6]:
soup = BeautifulSoup(response.text, 'html.parser')
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

## Extract all column/variable name from HTML table header

In [7]:
html_tables = soup.find_all('table')
#html_tables

Starting from the third table is our target table that contains the actual launch records.

In [8]:
#Print out the third_tabel and check its content
first_launch_table = html_tables[2]
#print(first_launch_table)

The column names are embedded in the table header elements `th`. We iterate through the `<th>` elements and apply the provided `extract_column_from_header()` function to extract column name one by one.

In [9]:
column_names = []
all_cols = first_launch_table.find_all('th')
for header in all_cols:
    col_name = extract_column_from_header(header)
    if col_name is not None and len(col_name) > 0:
        column_names.append(col_name)

In [10]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## Create a dataframe by parsing the launch HTML tables

Create an empty dictionary with keys from the extracted column names from this extraction. Later, this dictionary will be converted into a dataframe.

In [11]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.

Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.

To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:

In [12]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        cells=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            launch_dict['Flight No.'].append(flight_number)
            
            # Date value
            datatimelist=date_time(cells[0])
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(cells[1])
            if not(bv):
                bv=cells[1].a.string
            #print(bv)
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = cells[2].a.string
            #print(launch_site)
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = cells[3].a.string
            #print(payload)
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(cells[4])
            #print(payload)
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = cells[5].a.string
            #print(orbit)
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            if cells[6].a is not None:
                customer = cells[6].a.string
                launch_dict['Customer'].append(customer)
            else:
                launch_dict['Customer'].append(None)
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(cells[7].strings)[0]
            #print(launch_outcome)
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(cells[8])
            #print(booster_landing)
            launch_dict['Launch outcome'].append(launch_outcome)
            

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.

In [13]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [14]:
df.Customer.value_counts(dropna=False)

Customer
NaN                       121
NASA                       32
SpaceX                     30
Iridium Communications      8
SES                         5
USAF                        3
Telesat                     2
Thaicom                     2
CONAE                       2
NRO                         2
Sirius XM                   2
SKY Perfect JSAT Group      2
None                        2
ABS                         2
AsiaSat                     2
Orbcomm                     2
Türksat                     1
USSF                        1
Republic of Korea Army      1
U.S. Space Force            1
Sky Perfect JSAT            1
Spacecom                    1
Canadian Space Agency       1
PSN                         1
Spaceflight Industries      1
Es'hailSat                  1
Hisdesat                    1
Telkom Indonesia            1
Thales-Alenia               1
Hispasat                    1
Northrop Grumman            1
MDA                         1
SES S.A.                    1
N

In [15]:
#Store it as csv
df.to_csv('spacex_web_scraped.csv', index=False)